### Correction of the ML-Ready Cell-Painting Data Sets
#### Prerequisite
This notebook is part of the master thesis of Luis Vollmers. It uses the cell paintin data set of Bray et al. and the goal of the project is to predict pubchem assay data. Originally this notebook was concerned with pubchem AID 1030 but this might be subject to change.
#### Introduction
The datasets generated in ml_ready of step 01 comprise several mistakes that have been made and that need to be corrected within this jupyter notebook. Mostly, the compounds listed in the rows of the ML-ready dataframes are not unique, due to a misunderstanding stemming from the Bray et al Cell-Painting paper. It was stated that the Metadata_broad_sample is a unique identifier which was found to be wrong. Identical compounds which were used in different concentrations and/or on different well-plates were assigned different Metadata_broad_sample values. This needs to be corrected as well as the averaging over the different column values. In the data columns, the cell painting features are listed and so far the algorithm just kept the first concentration and took the average of the respective multiplicates. The correct way of doing it however is to check how many concentrations are present and then take the median of the concentration that is most frequent which is done by this jupyter notebook. 
#### Summary of the steps in this notebook
1. Import the ML-ready pubchem-assay and the preprocessed cell-painting raw data
2. Reintroduce the center median data into the pubchem df
3. Treat the multi-concentration Compounds adequately
4. Export the output


In [19]:
import pandas as pd

#### 1. Import the ML-ready pubchem-assay and the preprocessed cell-painting raw data
- Inputs are taken from the directory for step 1 of the pubchem assays and from the preprocessing directory
- the data from cp_1030 is erroneous but the meta data is needed for the overlap with the center median data
- therefore filter the df for the metadata

In [53]:
# load the data into RAM 'cp_1458' can be used as a variable for a bash script
dataset_path='../../01-FilteringAssays/ml_ready/cp_1458.csv'
df = pd.read_csv(dataset_path, index_col=0).drop(axis=1,columns=['Unnamed: 0.1'])
# read the cell-painting data into RAM
cp = pd.read_csv('../../../preprocessing/cp_center_median.csv', index_col=0)

# define the meta cols of the pubchem assays that are relevant and filter the df 
meta_cols = ['Metadata_broad_sample','PUBCHEM_ACTIVITY_OUTCOME', 'PUBCHEM_ACTIVITY_SCORE']
df = df.loc[:,meta_cols]

In [21]:
df

,Metadata_broad_sample,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE
0,BRD-A00037023-001-05-1,Inactive,0.0
1,BRD-A00051892-001-05-0,Inactive,0.0
2,BRD-A00430899-001-05-9,Inactive,0.0
3,BRD-A00489811-001-05-0,Inactive,0.0
4,BRD-A00520476-001-03-3,Inactive,0.0
...,...,...,...
7165,BRD-K99995088-001-05-2,Inactive,0.0
7166,BRD-M01343043-001-05-4,Inactive,0.0
7167,BRD-M57109082-001-05-4,Inactive,0.0
7168,BRD-M59199604-001-05-2,Inactive,0.0


#### 2. Reintroduce the Center Median Data into the Pubchem DF
- the common column is Metadata_broad_sample which was responsible for the errors in the first place
- it is actually the only identifier column that is left in the ML-ready df
- the inner merge makes sure that only rows are kept that can be found in both dataframes

In [36]:
# merge the cp and the assay data on the common column
merged = pd.merge(left=cp, right=df, on='Metadata_broad_sample', how='inner')

In [37]:
merged

,Metadata_broad_sample,CPD_SMILES,Metadata_Plate,Metadata_Plate_Map_Name,Metadata_ASSAY_WELL_ROLE,Metadata_mmoles_per_liter,CAN_SMILES,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,...,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0,SINGLE_CONC,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE
0,BRD-K78414110-001-02-8,COc1ccc2c(c1)occ(CC(O)=O)c2=O,24278,H-BIOA-007-3,treated,5.000000,COc1ccc2c(=O)c(CC(=O)O)coc2c1,259.863281,-12.617188,-2.617188,...,-0.123436,-0.234958,-0.261659,-0.269276,0.162881,0.061376,0.104751,True,Inactive,0.0
1,BRD-K78485176-001-02-9,CCCc1nc(C(C)(C)O)c(C(=O)OCc2oc(=O)oc2C)n1Cc1cc...,24278,H-BIOA-007-3,treated,5.000000,CCCc1nc(C(C)(C)O)c(C(=O)OCc2oc(=O)oc2C)n1Cc1cc...,187.695312,-3.902344,-6.980469,...,-0.227765,-0.182558,-0.212246,-0.204336,0.087412,0.048356,0.082538,True,Inactive,0.0
2,BRD-K78612426-001-02-6,COc1cc(OC)c2ccc(=O)oc2c1,24278,H-BIOA-007-3,treated,5.000000,COc1cc(OC)c2ccc(=O)oc2c1,314.210938,-8.851562,-5.964844,...,-0.131496,-0.153068,-0.156526,-0.125545,0.107378,0.023594,0.045900,True,Inactive,0.0
3,BRD-K78637815-001-01-4,COc1ccc(cc1)-c1oc2cc(OC)ccc2c1C(=O)c1ccc(cc1)C#N,24278,H-BIOA-007-3,treated,5.000000,COc1ccc(-c2oc3cc(OC)ccc3c2C(=O)c2ccc(C#N)cc2)cc1,277.261719,-22.066406,0.042969,...,-0.138155,-0.128052,-0.116322,-0.113404,0.232823,0.171553,0.216743,True,Inactive,0.0
4,BRD-K78643075-001-03-3,OC1(CCN(CCCC(=O)c2ccc(F)cc2)CC1)c1ccc(Br)cc1,24278,H-BIOA-007-3,treated,2.379167,O=C(CCCN1CCC(O)(c2ccc(Br)cc2)CC1)c1ccc(F)cc1,330.347656,-7.035156,-0.277344,...,-0.048945,-0.091941,-0.096263,-0.109370,0.053506,0.038483,0.040109,True,Inactive,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7065,BRD-K92683369-001-07-7,Cn1c(cc(=O)c2ccccc12)-c1ccc2OCOc2c1,25580,C-2113-01-D39-026,treated,5.000000,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,-498.812500,17.882812,-6.312500,...,-0.005104,0.010366,0.044850,0.029993,0.036641,0.025382,-0.000969,False,Inactive,0.0
7066,BRD-K92683369-001-07-7,Cn1c(cc(=O)c2ccccc12)-c1ccc2OCOc2c1,25581,C-2113-01-D39-026,treated,5.000000,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,-745.437500,-10.445312,11.953125,...,0.132028,0.140386,0.167078,0.113069,-0.105950,-0.045796,-0.083033,False,Inactive,0.0
7067,BRD-K92683369-001-07-7,Cn1c(cc(=O)c2ccccc12)-c1ccc2OCOc2c1,25581,C-2113-01-D39-026,treated,5.000000,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,-745.437500,-10.445312,11.953125,...,0.132028,0.140386,0.167078,0.113069,-0.105950,-0.045796,-0.083033,False,Inactive,0.0
7068,BRD-K92683369-001-07-7,Cn1c(cc(=O)c2ccccc12)-c1ccc2OCOc2c1,25583,C-2113-01-D39-026,treated,5.000000,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,-392.523438,-6.695312,-14.437500,...,-0.159918,-0.072812,-0.049299,-0.046058,-0.047889,-0.052204,-0.058999,False,Inactive,0.0


In [58]:
# this cell calculates the number of rows expected from the final dataframe
multi_list = []
concs_list = merged.CAN_SMILES.value_counts().to_list()

for i in concs_list:
    if i > 1:
        multi_list.append(i)
        
merged.shape[0]-sum(multi_list)+len(multi_list)

6415

#### 3. Treat the multi-concentration Compounds adequately
- this step is the only one that needs to be conducted manually
- first look at the multi conc compounds and determine how to treat them
- in this case the concentrations that are less frequent are dropped and the remaining ones are calculated into median
- for that purpose the merged DF is split into data and meta columns and only the data columns are treated accordingly
- the groupby method is used to calculate the compound wise median 
- the single concentrations are already calculated so only the multi concs are actually computed herein

In [24]:
# have a look at the multi concentration compounds and figure out how to treat them
merged.query('SINGLE_CONC==False').loc[:,['CAN_SMILES','Metadata_mmoles_per_liter']] # only uncomment if you havent checked for said compounds yet

,CAN_SMILES,Metadata_mmoles_per_liter
7010,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7011,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7012,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7013,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7014,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7015,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7016,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7017,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7018,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7019,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507


In [25]:
merged = merged.drop(merged[(merged.SINGLE_CONC==False) & (merged.Metadata_mmoles_per_liter > 4.0)].index)

In [26]:
# Quality Control step that makes sure only the relevant rows with the most concentrations are kept
merged.query('SINGLE_CONC==False').loc[:,['CAN_SMILES','Metadata_mmoles_per_liter']] # just needed as a one time lookup

,CAN_SMILES,Metadata_mmoles_per_liter
7010,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7011,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7012,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7013,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7014,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7015,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7016,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7017,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7018,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507
7019,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,3.580507


In [27]:
# define a list of the all columns of the merged data frame
all_cols = merged.columns.to_list()
# redefine meta columns with the meta information of the cell painting assay
meta_cols = ['CAN_SMILES','CPD_SMILES','Metadata_broad_sample','Metadata_Plate_Map_Name','Metadata_ASSAY_WELL_ROLE','Metadata_Plate','SINGLE_CONC','PUBCHEM_ACTIVITY_SCORE','PUBCHEM_ACTIVITY_OUTCOME']

In [28]:
# data cols are basically all columns without the meta data. hence the forloop that removes those from data_cols
data_cols = all_cols
for item in meta_cols:
    data_cols.remove(item)
    
# afterwards the 'CAN_SMILES' column is inserted at the first position
data_cols.insert(0,'CAN_SMILES')

In [29]:
# the merged data frame is split into two dataframes containing meta and raw data information
merged_data = merged.loc[:,data_cols]
merged_meta = merged.loc[:,meta_cols]

In [30]:
# this command takes the compound wise median of the data
merged_data = merged_data.groupby('CAN_SMILES').median().reset_index()

In [35]:
merged_data

,CAN_SMILES,Metadata_mmoles_per_liter,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,Brc1c(Br)c(Br)c2[nH]nnc2c1Br,5.0,99.679688,-1.738281,11.632812,-0.012594,-0.002082,0.0,0.009814,0.011497,...,0.206057,0.027947,0.060647,0.024703,0.030289,0.070795,0.070235,-0.026796,0.007465,-0.005555
1,Brc1c(CSc2nc3ccccc3s2)nc2ncccn12,5.0,-196.503906,14.769531,13.367188,0.022452,0.012719,0.0,-0.011546,-0.018795,...,-0.271055,0.064555,-0.006663,0.037034,-0.029089,-0.032247,-0.005805,-0.264220,-0.203184,-0.218126
2,Brc1c(NC2=NCCN2)ccc2nccnc12,5.0,210.992188,8.984375,-6.285156,0.031261,0.015688,0.0,-0.005825,0.004874,...,-0.077125,-0.059831,-0.055991,-0.029520,-0.075991,-0.097967,-0.104842,0.173642,0.103963,0.125708
3,Brc1cc(Br)c2nc(-c3cccs3)ccc2c1,5.0,-250.296875,14.230469,15.363281,-0.000253,0.007498,0.0,0.005429,0.015187,...,-0.127135,0.036756,-0.078057,-0.067460,-0.002717,-0.012557,0.007323,-0.004406,-0.023417,0.001108
4,Brc1ccc(C2NCCS2)cc1,5.0,-198.328125,-24.976562,-12.843750,0.012575,0.003766,0.0,-0.001987,0.005923,...,-0.095787,-0.005824,-0.015738,-0.021735,-0.043152,-0.054731,-0.073059,-0.067095,-0.009859,-0.012095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410,c1csc(-c2ncc3ccccn23)c1,5.0,94.054688,-0.242188,-3.851562,0.027352,0.017209,0.0,-0.009259,-0.010964,...,-0.017883,0.084464,0.033589,0.026292,-0.111741,-0.110463,-0.106537,0.027059,-0.009336,0.007035
6411,c1csc(-c2nn3c(Cn4nnc5ccccc54)nnc3s2)c1,5.0,181.953125,2.070312,14.453125,0.032925,0.018734,0.0,-0.003536,0.002948,...,0.115798,-0.019399,-0.019467,-0.026341,-0.040923,-0.084713,-0.048301,0.016904,-0.010133,-0.003711
6412,c1csc(C2C3=C(Nc4ncnn42)c2ccccc2OC3)c1,5.0,389.335938,-7.574219,-15.019531,0.032398,0.019976,0.0,-0.010679,0.033830,...,-0.162798,-0.108216,-0.123740,-0.121124,0.078535,0.051194,0.059726,-0.137802,-0.157153,-0.158237
6413,c1csc(CNc2nc(-c3cccs3)nc3ccccc23)c1,5.0,-151.742188,-6.214844,-3.714844,-0.005343,-0.002436,0.0,0.000586,-0.008357,...,-0.022661,-0.019592,-0.074581,-0.051087,-0.078124,-0.172940,-0.147702,-0.039133,-0.037625,-0.039854


#### 4. Export the Output
- as a last step the only thing that needs to be done is to merge the meta and data columns back into one DF
- a bit of a clean up needs to be done since the merge command creates suplicates, which can be safely deleted
- output in csv format named according to the pubchem AID

In [31]:
# the median data is merged back with the meta data
merged = pd.merge(left=merged_meta, right=merged_data, on='CAN_SMILES', how='left')

In [32]:
# merging generally keeps all rows in both frames so that duplicates are generated, which get hereby deleted
merged = merged.drop_duplicates(subset='CAN_SMILES')

In [33]:
merged.to_csv('../_output/cp_1458.csv',index=False)

In [59]:
pd.read_csv('../_output/cp_1458.csv') # only uncomment for quality control purposes, i.e. visual conformation

,CAN_SMILES,CPD_SMILES,Metadata_broad_sample,Metadata_Plate_Map_Name,Metadata_ASSAY_WELL_ROLE,Metadata_Plate,SINGLE_CONC,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_OUTCOME,Metadata_mmoles_per_liter,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,COc1ccc2c(=O)c(CC(=O)O)coc2c1,COc1ccc2c(c1)occ(CC(O)=O)c2=O,BRD-K78414110-001-02-8,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,0.028752,-0.141545,-0.091271,-0.123436,-0.234958,-0.261659,-0.269276,0.162881,0.061376,0.104751
1,CCCc1nc(C(C)(C)O)c(C(=O)OCc2oc(=O)oc2C)n1Cc1cc...,CCCc1nc(C(C)(C)O)c(C(=O)OCc2oc(=O)oc2C)n1Cc1cc...,BRD-K78485176-001-02-9,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,-0.046285,-0.177514,-0.196004,-0.227765,-0.182558,-0.212246,-0.204336,0.087412,0.048356,0.082538
2,COc1cc(OC)c2ccc(=O)oc2c1,COc1cc(OC)c2ccc(=O)oc2c1,BRD-K78612426-001-02-6,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,-0.061549,-0.133824,-0.130796,-0.131496,-0.153068,-0.156526,-0.125545,0.107378,0.023594,0.045900
3,COc1ccc(-c2oc3cc(OC)ccc3c2C(=O)c2ccc(C#N)cc2)cc1,COc1ccc(cc1)-c1oc2cc(OC)ccc2c1C(=O)c1ccc(cc1)C#N,BRD-K78637815-001-01-4,H-BIOA-007-3,treated,24278,True,0.0,Inactive,5.000000,...,0.212673,-0.153656,-0.124610,-0.138155,-0.128052,-0.116322,-0.113404,0.232823,0.171553,0.216743
4,O=C(CCCN1CCC(O)(c2ccc(Br)cc2)CC1)c1ccc(F)cc1,OC1(CCN(CCCC(=O)c2ccc(F)cc2)CC1)c1ccc(Br)cc1,BRD-K78643075-001-03-3,H-BIOA-007-3,treated,24278,True,0.0,Inactive,2.379167,...,0.208695,-0.024429,-0.044756,-0.048945,-0.091941,-0.096263,-0.109370,0.053506,0.038483,0.040109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410,COc1ccc(NC2=C(NCCO)C(=O)c3ccccc3C2=O)cc1,COc1ccc(NC2=C(NCCO)C(=O)c3ccccc3C2=O)cc1,BRD-K24060909-001-06-0,C-2113-01-D39-007,treated,26204,True,0.0,Inactive,5.000000,...,-0.269482,0.125242,0.082472,0.112903,0.002772,-0.002557,0.013881,-0.390370,-0.302925,-0.305797
6411,O=C(Nc1ccc2ccccc2c1)[C@@H]1C[C@H]1[C@@H](NP(=O...,O=C(Nc1ccc2ccccc2c1)[C@@H]1C[C@H]1[C@@H](NP(=O...,BRD-K51635174-001-05-5,C-2113-01-D39-007,treated,26204,True,0.0,Inactive,5.000000,...,0.092705,-0.013763,-0.061586,-0.047562,-0.104460,-0.093003,-0.103421,-0.124817,-0.130828,-0.126630
6412,CCOC(=O)c1c(C)oc2c1c(C(c1cccnc1)N1CCCCC1)c(O)c...,CCOC(=O)c1c(C)oc2c1c(C(N1CCCCC1)c1cccnc1)c(O)c...,BRD-A43050491-001-05-6,C-2113-01-D39-007,treated,26204,True,0.0,Inactive,5.000000,...,0.003111,0.057371,0.013895,0.005887,-0.003589,0.081811,0.062094,-0.116301,-0.099438,-0.107922
6413,Cn1c(-c2ccc3c(c2)OCO3)cc(=O)c2ccccc21,Cn1c(cc(=O)c2ccccc12)-c1ccc2OCOc2c1,BRD-K92683369-001-03-6,H-BIOA-007-3,treated,24278,False,0.0,Inactive,3.580507,...,-0.104311,0.005627,-0.037484,-0.027687,0.023850,-0.032037,0.004835,-0.078895,-0.036355,-0.064550
